In [ ]:
cd ..

In [1]:
from final_utils import *
from sklearn.utils import shuffle

In [2]:
SC_CE_KLD = tf.keras.losses.SparseCategoricalCrossentropy()

In [3]:
gpu = tf.config.list_physical_devices('GPU')


tf.config.experimental.set_memory_growth(gpu[0], True)

2022-11-03 20:10:59.317307: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 20:10:59.340970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 20:10:59.343844: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
print(gpu)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
model_gap = model_inceptionresnet_multigap()
model_cnn = tf.keras.Sequential([hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1", trainable=False)])

2022-11-03 20:10:59.372996: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 20:10:59.373351: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 20:10:59.374109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 20:10:59.374774: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [6]:
root_path = generate_root_path() 

In [7]:
# BOOL PARAMETER TO SPECIFY IF THE TRAINING IS SPLITTED OR JOINT

joint = False


In [8]:
features_good1 = []
feats_MG = 'original'  
feats_CNN = 'border_600x600'
# feats_CNN_MG_PCA = 'cnn_mg_concat/pca_9744_auto'
cnn = 'cnn_efficientnet_b7'


def load_for_current_bad(i=0):
    main_path=root_path + 'Data/AesthAI/alm/splitted/alm_train/' 
    feats_MG = 'original' 
    feats_CNN = 'border_600x600'
    cnn = 'cnn_efficientnet_b7'

    np.random.seed(i)
    alm_train_bad = open(f'{main_path}data_bad{i+1}.json')
    bad_data = json.load(alm_train_bad)
    features_bad_list = []
    for data in bad_data:
        feat_path_1 = main_path + f'features/multigap/{feats_MG}/' + data['feature']
        feat_path_2 = main_path + f'features/{cnn}/{feats_CNN}/' + data['feature']
        connected = np.concatenate((np.squeeze(np.load(feat_path_1)), np.squeeze(np.load(feat_path_2))))
        features_bad_list.append(connected)
    features_bad_list = np.squeeze(np.array(features_bad_list))
    return features_bad_list


def loading_all_data_from_json(main_path = root_path + 'Data/AesthAI/alm/splitted/alm_train/', joint=False):
    features_good1 = []
    features_bad_list = []
    lbl = []
    all_data = []   
    feats_MG = 'original' 
    feats_CNN = 'border_600x600'
    cnn = 'cnn_efficientnet_b7'

    features_bad_list = list(map(load_for_current_bad,range(7)))

        
    alm_train_good = open(f'{main_path}/data_good1.json')
    good_data = json.load(alm_train_good)
    for data in good_data:
        feat_path_1 = main_path + f'features/multigap/{feats_MG}/' + data['feature']
        feat_path_2 = main_path + f'features/{cnn}/{feats_CNN}/' + data['feature']
        connected = np.concatenate((np.squeeze(np.load(feat_path_1)), np.squeeze(np.load(feat_path_2))))
        features_good1.append(connected)
    features_good1 = np.squeeze(np.array(features_good1))
    features_bad_list[0], features_bad1_val = extract_static_val_data(features_bad_list[0], perc = 0.11)
    features_good, features_good_val = extract_static_val_data(features_good1, perc = 0.11)
    
    
    val_data = np.concatenate((features_bad1_val, features_good_val), axis=0)
    val_lbl = np.concatenate((np.zeros(len(features_bad1_val)), np.ones(len(features_good_val))), axis=0 )
    val_data, val_lbl = shuffle(val_data, val_lbl)
    
    for i in range(7):
        data_i = np.concatenate((features_good, features_bad_list[i]) , axis=0)
        lbl_i = np.concatenate((np.ones(len(features_good)), np.zeros(len(features_bad_list[i]))), axis=0 )
        data_i, lbl_i = shuffle(data_i,lbl_i)
        all_data.append(data_i)
        lbl.append(lbl_i)

    all_data = np.array(all_data, dtype=object)
    lbl = np.array(lbl, dtype=object)
    
    if joint:
        clean_data = all_data[0]
        for i in range(1, 7):
            clean_data = np.vstack((clean_data, all_data[i]))
        
        clean_lbl = np.array([lbl[0]]).T
        for i in range(1, 7):
            clean_lbl = np.vstack((clean_lbl, np.array([lbl[i]]).T))
            
        return clean_data, clean_lbl, val_data, val_lbl
    
    return all_data, lbl, val_data, val_lbl  

In [9]:
# data_all = loading_all_data_from_json()
all_data, all_lbl, val_data, val_lbl = loading_all_data_from_json(joint=joint)

In [10]:
all_data[6].shape

(21182, 19488)

In [11]:

#dummy
def create_dummy(joint=joint):
    
    if not joint:
        dummy_all_data = []
        dummy_val_data = []
        for i in range(7):
            dummy_all_data.append(all_data[i][:, :100])
        dummy_val_data = val_data[:, :100]

    else:
        dummy_all_data = all_data[:, :100]
        dummy_val_data = val_data[:, :100]
    
    return dummy_all_data, dummy_val_data




# all_data, val_data = create_dummy(joint=False) # IF YOU WANT TO USE DUMMY DATA
# print(dummy_val_data.shape)

In [12]:
all_data[0].shape

(19364, 19488)

In [13]:
good_imgs_path = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark', 'images', 'good', '*'))
bad_imgs_path = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark', 'images', 'bad', '*'))

good_imgs_path_2 = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark2', 'images', 'good', '*'))
bad_imgs_path_2 = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark2', 'images', 'bad', '*'))

paths_bench = good_imgs_path + bad_imgs_path
labels_bench = np.concatenate((np.ones(len(good_imgs_path)), np.zeros(len(bad_imgs_path))))

paths_bench_2 = good_imgs_path_2 + bad_imgs_path_2
labels_bench_2 = np.concatenate((np.ones(len(good_imgs_path_2)), np.zeros(len(bad_imgs_path_2))))

# Train funcs

In [14]:
# def get_train_pairs(features_bad, features_good, train_size=0.9, shuffle=True):
#     train_data_bad = features_bad
#     print('train_data_bad shape', train_data_bad.shape)
#     train_data_good = features_good
#     print('train_data_good shape', train_data_good.shape)

#     input_data = np.concatenate( (train_data_bad,train_data_good),axis=0)
#     target_data = np.concatenate( (np.zeros(train_data_bad.shape[0]),np.ones(train_data_good.shape[0])) , axis=0 )
    
#     X_train, Y_train = shuffle(input_data,target_data)
#     return X_train, Y_train



def take_from_feats(data, pop):
    idx = np.nonzero(pop)
    new_data = np.squeeze(data[:, idx])
    return new_data


def calc_acc(model, weights_path, X_test, y_test, batch_size):
    '''
    Compares Max classes with targets, getting mean class precision
    '''
    
    model.load_weights(weights_path)
    y_pred = model.predict(X_test, batch_size=batch_size, verbose=0)
    acc = (np.argmax(y_pred,axis=-1) == y_test).mean()
    return acc


def trainer(model, data, weights_path, data_val, batch_size=128, epochs=30, learning_rate=0.03):
    X_train, y_train = data
    model.compile(loss=SC_CE_KLD,
                  optimizer=keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, 
                                                  epsilon=1e-07, decay=0, amsgrad=False))
    model.load_weights(weights_path) 

    checkpoint = keras.callbacks.ModelCheckpoint(weights_path, 
                                                 monitor='val_loss', 
                                                 verbose=0, 
                                                 save_best_only=True, 
                                                 mode='min')
    
    
    schedule = tf.keras.callbacks.LearningRateScheduler(lr_exp_decay, verbose=0)
    
    
    callbacks_list = [checkpoint, schedule]
    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0,
                        callbacks=callbacks_list,
                        validation_data = data_val)

    return history


def training(data, data_val, joint=joint):
    # creating model, and weights
    if joint:
        model = fc_model_softmax(input_num=(data[0].shape)[1]) 
    else:
        model = fc_model_softmax(input_num=(data[0][0].shape)[1])
    weights_path = f'models/Softmax/MultiGap_CNN_GA/pygad_popsize_8_vectorsize_5000_1_0.hdf5'#before training need to add weights path
    model.save_weights(weights_path) #if we want to cancel learning and start from 0
#     model.load_weights(weights_path_pretrained)
#     weights_path = f'models/Softmax/MultiGap_CNN_GA/pygad_popsize_8_vectorsize_5000_1_0.hdf5'
    
    epochs = 15
    batch_size = 128
    learning_rate = 0.003
    
    if joint:
        history = trainer(model, data, weights_path, data_val, batch_size, epochs, 
                      learning_rate=learning_rate)
        
        
    else:
        def trainer_i(i):
            history = trainer(model, (data[0][i], data[1][i]), weights_path, data_val, batch_size, epochs, 
                              learning_rate=learning_rate)


        list(map(trainer_i, range(7)))
    
        
    acc = calc_acc(model, weights_path, data_val[0], data_val[1], batch_size)
    return acc

# Eval funcs


In [15]:
def predict(x, y=None, model_gap=None, model=None, model_cnn=None, curr_pop=None):
    '''
    Does prediction on given numpy image using
    model_gap and model
    '''
    try:
        feat_mg = model_gap.predict(x, verbose=0)
    except:
        x = x[None]
        feat_mg = model_gap.predict(x, verbose=0)
        
    if model_cnn:
        feat_cnn = model_cnn.predict(y, verbose=0)
        feat = np.concatenate((np.squeeze(feat_mg), np.squeeze(feat_cnn)))
        feat = feat[None]
    else:
        feat = feat_mg
        
    feat = take_from_feats(feat, curr_pop)[None]
        
    pred_score = model.predict(feat, verbose=0)

    return pred_score
    
def predict_from_path(model_gap, model, paths, curr_pop=None, resize_func=None, size=None, for_all=False, save_results=None, 
                      save_to=None, model_cnn=None):
    #always requires list of paths
    predicted = []
    
    for i, path in enumerate(paths):
        img_mg = read_img(path=path, resize_func=resize_func, size=size, for_all=for_all)
        img_cnn = None
        if model_cnn:
            img_cnn = read_img(path=path, resize_func=resize_add_border, size=(600, 600))
        pred_score = predict(img_mg, img_cnn, model_gap, model, model_cnn, curr_pop)
        predicted.append(pred_score)
    
    predicted = np.array(predicted)
    predicted = np.squeeze(predicted)
    
    if save_results:
        np.save(save_to, np.argmax(predicted, axis=-1))
        
    return predicted

def calc_acc_eval(labels, predicted):
    '''
    Calculating mean class error, e.g. predicted classes are 1vs0, 0vs0, 0vs0, 0vs0, then we have acc=0.25
    Inputs: 
        labels = target labels
        predicted = predicted binary probability distribution for the input
    Output:
        mean class error
    '''
    acc = np.sum(np.array(labels) == np.argmax(np.array(predicted), axis=1)) / len(labels)
    return np.round(acc * 100, 2)

# def num_wrong_pred(labels, predicted):
#     '''
#     Calculating mean class error, e.g. predicted classes are 1vs0, 0vs0, 0vs0, 0vs0, then we have acc=0.25
#     Inputs: 
#         labels = target labels
#         predicted = predicted binary probability distribution for the input
#     Output:
#         mean class error
#     '''
#     wrong = len(labels) - np.sum(np.array(labels) == np.argmax(np.array(predicted), axis=1))
#     return wrong


In [16]:
# BOOL PARAMETER FOR FITNESS_FUNCTION!!!



def is_similar(population):
    pop_i = np.copy(population)
    idx_nan = np.where(pop_i == np.zeros(shape=(np.array(pop_i).shape[0], np.array(pop_i).shape[1])))
    pop_i[idx_nan[0], idx_nan[1]] = 5 # Hardcode
    is_sim = (population[None] == pop_i[:, None]).mean(axis=-1)
    idx = np.triu_indices(is_sim.shape[0])
    idx1 = idx[0]
    idx2 = idx[1]
    new_idx = np.where((idx1 == idx2) == False)
    idx1 = idx1[new_idx[0]]
    idx2 = idx2[new_idx[0]]
    idx = idx1, idx2
    sim_pop_idxs = np.where(is_sim[idx] >= 0.8) # threshold
    pop1 = idx1[sim_pop_idxs[0]]
    pop2 = idx2[sim_pop_idxs[0]]
    
    return pop1, pop2

def mutation_for_par(state, count_of_mut=100, similar_par=None):
    indx_0 = np.where(state == 0)[1]
    indx_1 = np.where(state == 1)[1]
    if similar_par is not None:
        idx_sim_1 = np.where(similar_par == 1)[1]
        conf_matrix = indx_0[:, None] == idx_sim_1[None]
        idx_to_del = np.where(conf_matrix)[0]
        indx_0 = np.delete(indx_0, idx_to_del)
        if len(indx_0) < count_of_mut:
            print('Not mutated')
            return state
    mutate_idxs_0 = np.random.choice(indx_0, size=count_of_mut, replace=False)
    mutate_idxs_1 = np.random.choice(indx_1, size=count_of_mut, replace=False)
    state[0, mutate_idxs_0] = 1
    state[0, mutate_idxs_1] = 0
    
    return state

def mutation(offspring, ga_inctance):
    idx = is_similar(offspring)
    idx1 = np.unique(idx[0])
    idx2 = np.unique(idx[1])
    for idx_1, idx_2 in zip(idx1, idx2):
        offspring[idx_1] = mutation_for_par(state=(offspring[idx_1])[None], 
                                        count_of_mut=2, similar_par=(offspring[idx_2])[None])
    return offspring



def fitness_function(chromosome, pop_idx):
    print(np.sum(chromosome))
    indexes = chromosome

    if not joint:
        
        data_train = []# To avoid OVERWRITING DATASET

        for i in range(7):
            data_train.append(take_from_feats(all_data[i], indexes))
            
    else:
        
        data_train = take_from_feats(all_data, indexes)        

    data_val = take_from_feats(val_data, indexes)

    

    trainer = training((data_train, all_lbl), (data_val, val_lbl), joint=joint)

    model = fc_model_softmax(input_num=np.sum(indexes))
    weights_path = f'models/Softmax/MultiGap_CNN_GA/pygad_popsize_8_vectorsize_5000_1_0.hdf5'
    model.load_weights(weights_path)
    
    predicted1 = predict_from_path(model_gap=model_gap, model=model, paths=paths_bench, curr_pop=indexes, resize_func=resize_max, 
                              size=(996, 996), for_all=False, model_cnn=model_cnn)
    
    acc1 = calc_acc_eval(labels_bench, predicted1)
    print(f'Eval accuracy 1: {acc1} %')
#     num_wrong_pred1 = num_wrong_pred(labels_bench, predicted1)
    
    predicted2 = predict_from_path(model_gap=model_gap, model=model, paths=paths_bench_2, curr_pop=indexes, resize_func=resize_max, 
                              size=(996, 996), for_all=False, model_cnn=model_cnn)
    
    acc2 = calc_acc_eval(labels_bench_2, predicted2)
    print(f'Eval accuracy 2: {acc2} %')
#     num_wrong_pred2 = num_wrong_pred(labels_bench_2, predicted2)

    fx = 10*(100 - acc1)/25 + 10*(100 - acc2)/25 + (np.sum(indexes)/5000)
    
    best = np.load('best_res/best.npy')
    acc1_best = best[0]
    acc2_best = best[1] 
    
    acc = (acc1 + acc2) / 2
    print(f'fit score: {1/abs(7.5 - fx)}')
    
    if (acc1 > acc1_best and acc2 > acc2_best) or acc > 90:
        np.save(f'best_res/best_solution_2_{acc1}_{acc2}', indexes)
        model.save_weights(f'best_res/best_5k_2_{acc1}_{acc2}.hdf5')
#         acc1_best = acc1
#         acc2_best = acc2
#         best = np.array([acc1_best, acc2_best])
#         np.save('best_res/best_1_0', best)
    
    

    return 1/abs(7.5 - fx)





# Pygad

In [1]:
cd Genetic_Algorithm

[WinError 2] The system cannot find the file specified: 'Genetic_Algorithm'
C:\Users\CareAware\Consulting_Projects\aesthetic-ai\DeepFL\DeepFL\Genetic_Algorithm


In [17]:
from pygad import GA

In [ ]:
cd ..

In [18]:
def generate_init_pop(feat_vector_size, transformed_feat_vector_size, pop_size, init=False, init_size=3):
    # generates random population
    # INIT ONLY WHEN ORIGINAL DATA SHAPE! NOT DUMMY!
    k = transformed_feat_vector_size // pop_size
    path_list = glob(f'{root_path}/Projects/aesthetic-ai/DeepFL/DeepFL/best_res/best_solution*')
    random.shuffle(path_list)
    population = []
    if init:
        for i in range(init_size):
            population.append(np.load(path_list[i]))
        
    len_pop = len(population)
    
    for i in range(1, pop_size - len_pop+1):
        main_pop = np.concatenate((np.ones(transformed_feat_vector_size - i*k), np.zeros(feat_vector_size - transformed_feat_vector_size + i*k)))
        np.random.shuffle(main_pop)
        population.append(main_pop)
        
    return population

In [19]:
init_pop = generate_init_pop(19488, 5000, 12, init=True, init_size=3)

In [20]:
init_pop[11].sum()

1256.0

In [21]:
ga_instance = GA(num_generations=100,
                       num_parents_mating=8,
                       fitness_func=fitness_function,
                       initial_population=init_pop,
                       num_genes=len(init_pop[0]),
                       gene_type=int,
                       parent_selection_type="rws",
                       keep_parents=2,
                       crossover_type="two_points",
                       mutation_type=mutation,
                       mutation_by_replacement=True,
                       random_mutation_min_val=0,
                       random_mutation_max_val=1,
                       save_best_solutions=True)

/home/zaven/Projects/aesthetic-ai/DeepFL/DeepFL/pygad.py:581: UserWarning: The mutation_by_replacement parameter is set to True while the mutation_type parameter is not set to random but (<function mutation at 0x7f1a9c2a5820>). Note that the mutation_by_replacement parameter has an effect only when mutation_type='random'.
  if not self.suppress_warnings: warnings.warn("The mutation_by_replacement parameter is set to True while the mutation_type parameter is not set to random but ({mut_type}). Note that the mutation_by_replacement parameter has an effect only when mutation_type='random'.".format(mut_type=mutation_type))
/home/zaven/Projects/aesthetic-ai/DeepFL/DeepFL/pygad.py:820: UserWarning: Use the 'save_best_solutions' parameter with caution as it may cause memory overflow when either the number of generations or number of genes is large.
  if not self.suppress_warnings: warnings.warn("Use the 'save_best_solutions' parameter with caution as it may cause memory overflow when either t

In [ ]:

# CHANGE FIT SCORE FROM ACC TO FX!!!!!!!! --> Done
ga_instance.run()

4091


2022-11-03 20:13:03.721014: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-11-03 20:14:54.995961: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-11-03 20:14:56.444960: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-03 20:14:56.451208: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.2.300, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-11-03 20:14:56.452631: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-11-03 20:14:56.452661: W tensorflow/stream_executor/gpu/

Eval accuracy 1: 89.19 %
Eval accuracy 2: 88.2 %
fit score: 0.4233341800016931
5000


In [ ]:
solutions = ga_instance.best_solutions()


In [ ]:
ga_instance.best_solutions_fitness